## 웹 크롤링

### 인터넷 접속 라이브러리 추가



In [16]:
from urllib.request import urlopen, Request

# 도시별 날씨 검색함수
def get_weather(city):
    # 기상청 홈페이지 도시별 날씨 페이지
    url = 'https://www.weather.go.kr/w/obs-climate/land/city-obs.do'
    page = urlopen(url=url)

    text = page.read().decode('utf-8')
    text = text[text.find(f'>{city}</a>'):]

    # 기온 가져오기
    for i in range(7):
       text = text[text.find('<td>')+1:] 
       

    start = 3
    end = text.find('</td>')
    current_temp = text[start:end]
    print(f'{city}의 현재 기온은 {current_temp}˚C 입니다.')

# 습도 가져오기
    for i in range(3):
       text = text[text.find('<td>')+1:] 
       

    start = 3
    end = text.find('</td>')
    current_humid = text[start:end]
    print(f'{city}의 현재 습도는 {current_humid}% 입니다.')

get_weather('부산')


부산의 현재 기온은 7.1˚C 입니다.
부산의 현재 습도는 41% 입니다.


### OpenAPI 크롤링

- 공공데이터 포털
    - https://www.data.go.kr
    - 부산광역시 버스정보안내기 현황

In [21]:
from urllib.request import urlopen, Request
from urllib.parse import quote   # 한글을 URLencode 변환하는 함수 롯데 --> %EB%A1%AF%EB%8D%B0
import json

def getRequestUrl(url):
    req = Request(url)

    try:
        res = urlopen(req)
        if res.getcode() == 200:
            return res.read().decode('utf-8')
    except Exception as e:
        print(e)
        return None

# stationName 정류소이름
def getDataPortalSearch(stationName, type):
    api_url = 'https://apis.data.go.kr/6260000/BusanTblBusinfoeqStusService/getTblBusinfoeqStusInfo'
    api_url += '?serviceKey=Q3R5dAv8J37EqouPg97nKYcw2aCt2RsYHZpjzKERhKAminzU%2FKlCYlr59sCIBSNROEHCIMixAkdLEW7eRo8xpA%3D%3D'
    api_url += '&pageNo=1'
    api_url += '&numOfRows=10'
    api_url += f'&resultType={type}'
    api_url += f'&stationLoc={quote(stationName)}'

    response = getRequestUrl(api_url)

    if response == None:
        return None
    else:
        return json.loads(response)

getDataPortalSearch('롯데백화점', 'json')

quote('롯데')



<urlopen error [SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:992)>


'%EB%A1%AF%EB%8D%B0'

### BeautifulSoup (ver.4)

웹크롤링을 편하게 해주는 도구(라이브러리)

``` shell
pip installl beautifulsoup4
```

In [22]:
!pip install beautifulsoup4

     -------------------------------------- 129.4/129.4 kB 3.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 모듈 import

```python
from bs4 import BeautifulSoup
```



In [25]:
from bs4 import BeautifulSoup
import requests 

url = "https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4%EC%9B%B9%EB%A7%9D%EC%9B%90%EA%B2%BD"

response = requests.get(url)

if response.status_code == 200:  # webpage OK
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup)
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    print(title.get_text())
else:
    print(f'Error : {response.status_code}')



제임스웹 우주망원경 질문이요


In [28]:
from bs4 import BeautifulSoup
import requests 

url = "https://kin.naver.com/search/list.naver?query=%EC%A0%9C%EC%9E%84%EC%8A%A4%EC%9B%B9%EB%A7%9D%EC%9B%90%EA%B2%BD"

response = requests.get(url)

if response.status_code == 200:  # webpage OK
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup)
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    ul = soup.select_one('ul.basic1')
    titles = ul.select('li > dl > dt > a')
    for title in titles:
        print(title.get_text())
else:
    print(f'Error : {response.status_code}')

제임스웹 우주망원경 질문이요
제임스 웹 망원경의 종류 알려주세요
최고고도 부탁드립니다. 제임스웹 망원경이
제임스 웹 망원경 우주사진 원본 어디서... 
제임스 웹 우주망원경이 찍은 첫 사진은... 
제임스 웹 망원경
제임스 웹 망원경 촬영 원리
제임스웹망원경 촬영질문
제임스 웹 우주망원경
제임스웹 망원경
